In [12]:


import os
import csv
import datetime
import re
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import configparser

config = configparser.ConfigParser()

        
# Read the configuration file
config.read("config.ini")

skills = config.get("other", "skills", fallback="Default Value")
experience = config.get("other", "experience", fallback="3")
job_titles = config.get("other", "job_titles").split(",")
location = config.get("other", "location")

## options are 1, 3, 7, 15, 30 days | will take the max out of it automatically
job_age = max(config.get("other", "jobAge", fallback="").split(','))

## Applications that is not required to be applied for the respective Job Title !
exclude_keywords = config.get("other", "exclude_keywords", fallback="").split(",") 

my_info = dict(config.items("my_info"))
question_mappings = {
    "experience": experience,
    "first name": my_info.get        ("First", my_info.get("first", "")),
    "last name": my_info.get         ("Last", my_info.get("last", "")),
    "number": my_info.get            ("Number", my_info.get("number", "")),
    "phone": my_info.get             ("Number", my_info.get("number", "")),
    "contact": my_info.get           ("Number", my_info.get("number", "")),
    "website": my_info.get           ("Github", my_info.get("github", "")),
    "github": my_info.get            ("Github", my_info.get("github", "")),
    "current city": my_info.get      ("City", my_info.get("city", "")),
    "current location": my_info.get  ("City", my_info.get("city", "")),
    "expected annual": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected salary": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected ctc": my_info.get      ("ECTC", my_info.get("ectc", "")),
    "salary expectation": my_info.get("ECTC", my_info.get("ectc", "")),
    "join": my_info.get              ("notice_period", my_info.get("Notice Period", "")),
    "languages": my_info.get         ("languages", my_info.get("Languages", "")),
    "languages": my_info.get         ("languages", my_info.get("Languages", "")),
    "job change": my_info.get        ("job_change_reason", my_info.get("Job Change Reason", "")),
    "current ctc": my_info.get       ("CTC", my_info.get("ctc", "")),
    "current annual ctc": my_info.get("CTC", my_info.get("ctc", "")),
    "current ctc": my_info.get       ("CTC", my_info.get("ctc", "")),
    "current annual ctc": my_info.get("CTC", my_info.get("ctc", "")),
    "expected annual": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected salary": my_info.get   ("ECTC", my_info.get("ectc", "")),
    "expected ctc": my_info.get      ("ECTC", my_info.get("ectc", "")),
    "salary expectation": my_info.get("ECTC", my_info.get("ectc", "")),
    "notice period": my_info.get     ("notice_period", my_info.get("Notice Period", "")),
    "join": my_info.get              ("notice_period", my_info.get("Notice Period", "")),
    "when can you start": my_info.get("notice_period", my_info.get("Notice Period", "")),
    "when can you start": my_info.get("notice_period", my_info.get("Notice Period", "")),
    "university": my_info.get        ("university", my_info.get("University", "")),
    "school": my_info.get            ("university", my_info.get("School", "")),
    "institute": my_info.get         ("university", my_info.get("Institute", "")),
    "university": my_info.get        ("university", my_info.get("University", "")),
    "school": my_info.get            ("university", my_info.get("School", "")),
    "institute": my_info.get         ("university", my_info.get("Institute", "")),
    "skills": skills
}


applied_jobs = set()  

wait = WebDriverWait(driver, 10)

ctc_expectations = {
    1: "0to3",
    2: "3to6",
    3: "6to10",
    4: "10to15",
    5: "15to25",
    6: "25to50",
    7: "50to75",
}

location_to_city_type_gid = {
    1: {"mumbai": 134},
    2: {"chennai": 183},
    3: {"usa": 9044},
    4: {"Hyderabad": 17},
    5: {"bangalore": 97},
    6: {"delhi/NCR": 9508},
    7: {"Pune": 139},
    8: {"Kolkata": 232},
    9: {"Mumbai(All Areas)": 9509},
}



# Redirect SSL handshake error messages to nul (Windows)
if os.name == 'nt':
    sys.stderr = open(os.devnull, 'w')
# Redirect SSL handshake error messages to /dev/null (Linux/Mac)
if os.name == 'posix':
    sys.stderr = open('/dev/null', 'w')

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")

In [8]:
# Create a new instance of the Chrome web driver
driver = webdriver.Chrome(options=chrome_options)

def accept_cookies():
    actions = ActionChains(driver)
    actions.move_by_offset(1, 1).click().perform()
    
def refresh_page():
    driver.refresh()
    accept_cookies()  # Accept cookies after refreshing

def login_portal():
    
    # Navigate to the Naukri website
    driver.get("https://www.naukri.com/nlogin/login")
    accept_cookies()
    

    username = "tusharmalhan@gmail.com"
    password = "tusharmalhan@21"


    sleep(2)
    # Enter your login credentials
    driver.find_element(By.ID, "usernameField").send_keys(username)
    driver.find_element(By.ID, "passwordField").send_keys(password)
    driver.find_element(By.ID, "passwordField").send_keys(Keys.ENTER)

    sleep(4)

login_portal()


if re.search(r'Invalid', driver.page_source, re.IGNORECASE):
    driver.refresh()
    login_portal()
else:
    print('\nLogged in Successfully to Naukri Portal \n')


Logged in Successfully to Naukri Portal 



In [15]:

def get_element_xpath(element):
    return str(driver.execute_script("function absoluteXPath(element) {var comp, comps = [];var parent = null;var xpath = '';var getPos = function(element) {var position = 1, curNode;if (element.nodeType == Node.ATTRIBUTE_NODE) {return null;}for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {if (curNode.nodeName == element.nodeName) {++position;}}return position;};if (element instanceof Document) {return '/';}for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {comp = comps[comps.length] = {};switch (element.nodeType) {case Node.TEXT_NODE:comp.name = 'text()';break;case Node.ATTRIBUTE_NODE:comp.name = '@' + element.nodeName;break;case Node.PROCESSING_INSTRUCTION_NODE:comp.name = 'processing-instruction()';break;case Node.COMMENT_NODE:comp.name = 'comment()';break;case Node.ELEMENT_NODE:comp.name = element.nodeName;break;}}for (var i = comps.length - 1; i >= 0; i--) {comp = comps[i];xpath += '/' + comp.name.toLowerCase();if (comp.hasOwnProperty('position')) {xpath += '[' + comp.position + ']';}}return xpath;}return absoluteXPath(arguments[0]);", element))


In [16]:

def naukri_portal(experience, job_title, location, job_age, exclude_keywords):
    
    for positions in job_title:

        # import pdb;pdb.set_trace()
        job_title = '-'.join(positions.split(' ')).lower()
        job_url = f"https://www.naukri.com/{job_title}-jobs?experience={experience}"

        if location is None or location =="None":
            location_preference = [
                location_to_city_type_gid[4].get("Hyderabad"), 
                location_to_city_type_gid[5].get("bangalore"),
                location_to_city_type_gid[5].get("Mumbai(All Areas)")
                ]
            job_url = f"https://www.naukri.com/{job_title}-jobs?experience={experience}"
            for location in location_preference:
                if location:
                    job_url += f"&cityTypeGid={location}"
        else:
            job_url +="&wfhType=2"

        # Add the CTC expectations to the job URL
        ctc_expectations_values = [ctc_expectations[5], ctc_expectations[6], ctc_expectations[4]]
        for ctc_expectations_value in ctc_expectations_values:
            job_url += f"&ctcFilter={ctc_expectations_value}"

        job_url += f"&jobAge={job_age}"

    
        sleep(2)
        driver.get(job_url)
        accept_cookies()

        # Get the current window handle.
        current_window_handle = driver.current_window_handle
        sleep(1)
        
        
        try:
            job_links = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "list")))
            try:
                sleep(1)
                all_jobs = wait.until(EC.visibility_of_all_elements_located((By.TAG_NAME, "article")))
            except:
                all_jobs = [link.find_elements(By.TAG_NAME, "article") for link in job_links][0]
            print(f"\nLets initiate Automatic Job Search on Naukri Portal for {positions.upper()} position \n")
        
        except Exception as e :
            print(f"\n\t Apologies, No Jobs found for Position {positions.upper()} as per your requirement \n ")
            continue
    
        # Create a DataFrame to store the job data
        job_data = []

        for job in all_jobs:

            if job in applied_jobs:
                continue
    
            import pdb;pdb.set_trace()
            sleep(1)
        
            try:
                job.click()
                
            except:
                sleep(1)
                print("\nStaleElementReferenceException occurred in job click. Retrying...\n")
                import pdb;pdb.set_trace()
                sleep(2)
                try:
                    driver.refresh()
                    driver.get(job_url)
                    accept_cookies()
                    sleep(3.5)
                    job.click()
                except:
                    driver.quit()
                    print("\nCant proceed with the stale element\n")
                    df.to_csv(csv_file_path, index=False)

                    # Print a success message
                    print(f"\nJob data has been saved to {csv_file_path}\n")
                    
                    # job_data.append([job_title, '', f"Not Applying because of keyword mentioned {keyword.lower()}", '', '', "No URL", '', '', ''])
                    # driver.refresh()
                    # driver.get(job_url)
                    # accept_cookies()
                    # sleep(3.5)
                    # job.click()
                
            accept_cookies()
            sleep(2)
            ## Switch to the new tab.
            for window_handle in driver.window_handles:
                if window_handle != current_window_handle:
                    driver.switch_to.window(window_handle)
        
            ## Switch to the new tab one liner
            # [ driver.switch_to.window(window_handle) for window_handle in driver.window_handles if window_handle != current_window_handle ]

                

            try:
                try:
                    html_content = driver.find_element(By.CLASS_NAME, "job-desc").text
                    recruiters_details = re.findall(r'\+91\d{10}', html_content)
                    if not recruiters_details:
                        recruiters_details = driver.find_element(By.CLASS_NAME, "rec-details").text
                    else:
                        import pdb;pdb.set_trace()
                except: 
                    # print('Look for phone number in description ')
                    recruiters_details = ''.join(recruiters_details)
            except NoSuchElementException:
                recruiters_details = "No Recruiters Details Found"

                
                
                
            try:
                try:job_title = driver.find_element(By.CLASS_NAME, "jd-header-title").text
                except: job_title = driver.title
                apply_flag = True
                for keyword in exclude_keywords:
                    if keyword.lower() in [i.lower().replace('(', '').replace(')', '') for i in job_title.split(' ')]:
                        print('Not applying for:', job_title, f' as keyword "{keyword.lower()}" is mentioned in the job title\n')
                        job_data.append([job_title, '', f"Not Applying because of keyword mentioned {keyword.lower()}", '', '', "No URL", '', '', ''])
                        apply_flag = False
                        break

                if not apply_flag:
                    sleep(1)
                    driver.close()
                    driver.switch_to.window(current_window_handle)
                    sleep(1.5)
                    continue

                company_name_raw = driver.find_element(By.CLASS_NAME, "jd-header-comp-name")
                company_name = company_name_raw.find_element(By.TAG_NAME, "a").text
                salary = driver.find_element(By.CLASS_NAME, "salary").text
                print(f'\nClicking the Next Job Title {job_title}  Button...\t\n')
                job_data.append([job_title, company_name, "Applied Successfully", salary, recruiters_details ,"No URL", '','',''])
            
            except Exception as e :

                sleep(2)
                print(f"\nFound some issues in getting meta info from the job we need to click \n" )

                print("\n############## PORTAL TO APPLY DIRECTLY ON THE WEBSITE ############# \n")
                
                # import pdb;pdb.set_trace()

                company_name = driver.title.split('-')[1].split(',')[0]
                result = manual_apply(job_data, job_title, company_name)
                if result == "Career Section Unavailable":
                    job_data.append([job_title, company_name, "Career Section Unavailable", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                elif result == "Issue in Web Page":
                    job_data.append([job_title, company_name, "Issue in Web Page", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                else:
                    job_data.append(["job_title", "company_name", "Apply manually on company's portal", "  Not Disclosed by Recruiter ", recruiters_details, driver.current_url, '','',''])
                driver.close()
                driver.switch_to.window(current_window_handle)
                sleep(3) 
                continue
            
            ### Apply for the job        
            try:
                try:
                    link_text = driver.find_element(By.CLASS_NAME, "apply-button")
                except NoSuchElementException:
                    link_text = driver.find_element(By.CLASS_NAME, "already-applied")
                    print(f"\nAlready applied to {company_name} for the title {job_title}.\n ")
                    job_data.append([job_title, company_name, "Already applied", salary, recruiters_details, "No URL", '','',''])
                    driver.close()
                    driver.switch_to.window(current_window_handle)
                    continue

            except NoSuchElementException:
                print('\n############## PORTAL TO APPLY DIRECTLY ON THE WEBSITE ############# \n')
                
                try:
                    link_text = driver.find_element(By.CLASS_NAME, "company-site-button")
                    link_text.click()
                except:
                    # link_text = driver.find_element(By.CLASS_NAME, "walkin-button")
                    print('\nWalkin position\n')
                sleep(1.5)
                company_website = driver.current_url

                driver.close()
                # Switch to company website
                [ driver.switch_to.window(window_handle) for window_handle in driver.window_handles if window_handle != current_window_handle ]

                
            

                import pdb; pdb.set_trace()
                result = manual_apply(job_data, job_title, company_name)
                if result == "Career Section Unavailable":
                    job_data.append([job_title, company_name, "Career Section Unavailable", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                elif result == "Issue in Web Page":
                    job_data.append([job_title, company_name, "Issue in Web Page", "Not Disclosed by Recruiter", [], driver.current_url, '', '', ''])
                else:
                    job_data.append(["job_title", "company_name", "Apply manually on company's portal", "  Not Disclosed by Recruiter ", recruiters_details, driver.current_url, '','',''])
                driver.close()
                driver.switch_to.window(current_window_handle)
                sleep(3)
                continue
            
            sleep(1)
            
            print('Clicking Apply button... \t\n')
            
            
            try:    link_text.click()
            except: sleep(2.5);link_text.click()

            import pdb;pdb.set_trace()
            print(' clicking on Update and apply button \n\t')

            sleep(2)
            
            if 'This company does not require you to apply again as you have previously' in driver.page_source:
                print(f"\nJob submission failed for the title {job_title} to {company_name}, as company has blocked you\n")
                job_data.append([job_title, company_name, "Company Banned You, LOL ! ", salary, recruiters_details ,driver.current_url, '','',''])
                ### Switch back to original tab, as job cant be posted
                driver.close()
                driver.switch_to.window(current_window_handle)
                continue

            filling_questionnaire(company_name, job_data)

        
            ### After filling the Quesstionnaire form , if still you find questionnaire, issue in Page
            qus_form = None
            try:
                sleep(3)
                qus_form = driver.find_element(By.NAME, "qusForm")
            except:
                try:
                    qus_form = driver.find_element(By.NAME, "qupForm")
                except: ...
            if qus_form:
                print(f"\nApply Manually \nJob submission failed for the title {job_title} to {company_name}.\n")
                job_data.append([job_title, company_name, "Submission Failed ", salary, recruiters_details ,driver.current_url, '','',''])
                # import pdb;pdb.set_trace()
                driver.close()
                driver.switch_to.window(current_window_handle)
                continue
            
            sleep(2)
            try:success_element = driver.find_element(By.CLASS_NAME, "apply-status-header.green")
            except:...
            daily_quota_limit = None
            
            try:daily_quota_limit = driver.find_element(By.XPATH, "//span[contains(text(),'Your daily quota has been expired.')]")
            except NoSuchElementException: ...
            if daily_quota_limit:
                driver.close()
                print("\nThe limit to apply to all your specific jobs has been reached!")
                print('Catch yeah tommorrow')
                break
            sleep(1.2)
            if success_element:
                print(f"\nJob submitted successfully for the title {job_title} to {company_name}\n\n")
                sleep(1)
            else:
                try:
                    if link_text.text !='Applied':
                        # import pdb;pdb.set_trace()
                        print(f"Job submission failed for the title {job_title} to {company_name}.\n")
                        job_data.append([job_title, company_name, "Submission Failed ", salary, recruiters_details ,driver.current_url, '','',''])
                    else: ...
                except:
                    driver.refresh()
                    driver.back()

            # close the driver for current job portal
            driver.close()

            # add the job to the set
            applied_jobs.add(job)

            # Switch back to the original tab.
            driver.switch_to.window(current_window_handle)

        # Generate a timestamp as the sheet name
        timestamp = datetime.now().strftime("%d_%m__%H__%M%S")
    
        # Create the main directory
        main_directory = "Naukri_Jobs_cv"
        if not os.path.exists(main_directory):
            os.makedirs(main_directory)

        # Create the directory for the current date if it doesn't exist
        directory_name = datetime.now().strftime("%Y-%m-%d")
        directory_path = os.path.join(main_directory, directory_name)
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)

        # Construct the full file path with the directory and file name
        csv_file_path = os.path.join(directory_path, f"job_data_{timestamp}.csv")

        # Create a DataFrame from the job data
        df = pd.DataFrame(job_data, columns=["Job Title", "Company Name", "Status", "CTC", "Recruiters Details" ,"URL", "question_text", "input type", "user input text"])
        
    
        df.to_csv(csv_file_path, index=False)

        # Print a success message
        print(f"\nJob data has been saved to {csv_file_path}\n")

    driver.quit()

naukri_portal(experience, job_titles, location, job_age, exclude_keywords)







Lets initiate Automatic Job Search on Naukri Portal for PYTHON position 

> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(61)naukri_portal()

*** NameError: name 'o' is not defined
*** Newest frame
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(63)naukri_portal()

*** Newest frame
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(64)naukri_portal()

experience = '3'
job_title = 'python'
location = None
job_age = '7'
exclude_keywords = ['adobe', '3d', '2d', 'android', 'blockchain', 'dot', 'salesforce', 'frontend', 'angular', 'c++', 'game', ' azure', 'c#', 'java', 'UI', 'UX', 'react', 'reactjs', 'vue', 'ember', 'php', 'Applications', 'sales', 'asp', 'asp.net', '.net', 'flutter', 'C#.Net']
*** Newest frame
*** NameError: name 'o' is not defined
*** NameError: name 'o' is not defined
--Return--
None
> c:\users\tusharmalhan\appdata\local\temp\ipykernel_6132\124368119.py(115)naukri_portal()

UnboundLocalError: cannot access